In [ ]:
1

1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

In [ ]:
file_name = "/content/drive/MyDrive/DropBox Hackathon/LoL_Data/tournaments.json"

with open(file_name) as user_file:
    file = json.load(user_file)

In [ ]:
data = file.copy()

#### **Note: League Name is actually Tournament Name**

In [ ]:
# Initialize an empty list to store the extracted data
table_data = []

# Iterate through the JSON data and extract the required information
for league in data:
    league_id = league["leagueId"]
    league_name = league["name"]

    for stage in league["stages"]:
        stage_name = stage["name"]

        for section in stage["sections"]:
            section_name = section["name"]

            for match in section["matches"]:
                match_id = match["id"]
                match_state = match["state"]



                for game in match["games"]:

                    # print(game['state'])
                    game_id = game["id"]

                    if game['state'] == 'completed':

                        for team in game["teams"]:

                            team_id = team["id"]
                            team_result = team["result"]["outcome"]

                            # print(team_id, team_result)
                            table_data.append([league_id, league_name, stage_name, section_name, match_id, game_id, team_id, team_result])

                        # break

                    # Append the extracted data to the list



In [ ]:
len(table_data)

49256

In [ ]:
table_data[:3]

[['109518549825754242',
  '#2 Summer 2023',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446929',
  '109981650516317055',
  'win'],
 ['109518549825754242',
  '#2 Summer 2023',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446929',
  '110733889090164854',
  'loss'],
 ['109518549825754242',
  '#2 Summer 2023',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446930',
  '110733889090164854',
  'loss']]

In [ ]:
# Create a DataFrame from the extracted data
table_df = pd.DataFrame(table_data, columns=["League ID", "League Name", "Stage", "Section", "Match ID", "Game ID", "Team ID", "Outcome"])

# Display the final table
table_df.head()

,League ID,League Name,Stage,Section,Match ID,Game ID,Team ID,Outcome
0,109518549825754242,#2 Summer 2023,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win
1,109518549825754242,#2 Summer 2023,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss
2,109518549825754242,#2 Summer 2023,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss
3,109518549825754242,#2 Summer 2023,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win
4,109518549825754242,#2 Summer 2023,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win


In [ ]:
table_df.shape

(49256, 8)

In [ ]:
table_df["League Name"].value_counts()

Summer 2023                         5918
Spring 2023                         5112
Opening 2023                        1286
Closing 2023                         670
Split 1 2023                         638
                                    ... 
Midseason Streamathon 2020            18
LLA Promotion 2022                    14
Volcano League Super Finals 2022       8
Golden League Super Finals 2022        6
Stars League Super Finals 2022         6
Name: League Name, Length: 193, dtype: int64

In [ ]:
table_df["Stage"].value_counts()

Regular Season        32840
Playoffs               9468
Groups                 1834
knockouts              1378
Round 1                1314
Play In Groups          958
Round 2                 782
Play In Knockouts       236
Knockouts               108
Bracket Stage           102
Regional Finals         100
regional_qualifier       28
Promotion Series         24
play_in_knockouts        22
Regional Qualifier       20
east                     16
west                     16
promotion                10
Name: Stage, dtype: int64

In [ ]:
table_df.to_excel("game_level_data.xlsx")

In [ ]:
# table_df.info()

# Model 2
- Made `wins_pct`
- Ranking on `Reshma-Sahil` metric

In [ ]:
team_stats_list = []

for team_id, team_df in table_df.groupby('Team ID'):
    # print(team_id)

    # print(team_df)
    num_games = team_df["Game ID"].nunique()

    # print(num_games)

    num_wins = (team_df["Outcome"] == 'win').sum()
    num_loss = (team_df["Outcome"] == 'loss').sum()

    wins_pct = 100*(num_wins/num_games).round(4)
    # wins_pct.round(4)

    # print(num_wins, num_loss)

    team_stats_list.append([team_id, num_games, num_wins, num_loss, wins_pct])
    # break

In [ ]:
len(team_stats_list)

625

In [ ]:
team_stats = pd.DataFrame(team_stats_list, columns=["team_id", "num_games", "num_wins", "num_loss", "wins_pct"])

In [ ]:
# metric 1
team_stats['rs_metric'] = 0.25*team_stats['num_games'] + 0.75*team_stats['wins_pct']

team_stats_ranked = team_stats.sort_values(by ="rs_metric", ascending = False).reset_index(drop=True)
team_stats_ranked

,team_id,num_games,num_wins,num_loss,wins_pct,rs_metric,rs_metric_2
0,98767991853197861,512,334,178,65.23,176.9225,132.2455
1,100205573495116443,471,322,149,68.37,169.0275,128.7645
2,100725845018863243,462,307,155,66.45,165.3375,125.7825
3,99566404585387054,437,207,230,47.37,144.7775,105.8145
4,99566404579461230,398,209,189,52.51,138.8825,104.3335
...,...,...,...,...,...,...,...
620,99124844361058275,1,0,1,0.00,0.2500,0.1500
621,99124844362174905,1,0,1,0.00,0.2500,0.1500
622,105352091319919029,1,0,1,0.00,0.2500,0.1500
623,105351724613755142,1,0,1,0.00,0.2500,0.1500


In [ ]:
team_stats_ranked.describe()

,num_games,num_wins,num_loss,wins_pct,rs_metric
count,625.000000,625.000000,625.000000,625.000000,625.000000
mean,78.809600,39.404800,39.310400,43.956912,52.670084
std,82.253988,47.059915,38.609273,18.808198,28.297258
min,1.000000,0.000000,0.000000,0.000000,0.250000
25%,20.000000,7.000000,11.000000,33.330000,33.667500
50%,48.000000,21.000000,27.000000,45.970000,49.405000
75%,111.000000,57.000000,55.000000,55.380000,69.292500
max,512.000000,334.000000,235.000000,100.000000,176.922500


In [ ]:
# metric 2
team_stats['rs_metric_2'] = 0.15*team_stats['num_games'] + 0.75*team_stats['wins_pct']
team_stats_ranked = team_stats.sort_values(by ="rs_metric", ascending = False).reset_index(drop=True)
team_stats_ranked

,team_id,num_games,num_wins,num_loss,wins_pct,rs_metric,rs_metric_2
0,98767991853197861,512,334,178,65.23,176.9225,125.7225
1,100205573495116443,471,322,149,68.37,169.0275,121.9275
2,100725845018863243,462,307,155,66.45,165.3375,119.1375
3,99566404585387054,437,207,230,47.37,144.7775,101.0775
4,99566404579461230,398,209,189,52.51,138.8825,99.0825
...,...,...,...,...,...,...,...
620,99124844361058275,1,0,1,0.00,0.2500,0.1500
621,99124844362174905,1,0,1,0.00,0.2500,0.1500
622,105352091319919029,1,0,1,0.00,0.2500,0.1500
623,105351724613755142,1,0,1,0.00,0.2500,0.1500


In [ ]:
# team_stats_ranked = team_stats.sort_values(by = ["wins_ratio",  "num_games"], ascending = [False, False]).reset_index(drop=True)
# team_stats_ranked

,team_id,num_games,num_wins,num_loss,wins_ratio
0,104228817622043824,4,4,0,1.0
1,105357803054359120,2,2,0,1.0
2,105193385417106233,1,1,0,1.0
3,105193481874236222,1,1,0,1.0
4,105193554293818475,1,1,0,1.0
...,...,...,...,...,...
620,105301791701328923,1,0,1,0.0
621,105351724613755142,1,0,1,0.0
622,105352091319919029,1,0,1,0.0
623,99124844361058275,1,0,1,0.0


In [ ]:
def calc_rs_metric_by_bucket(row):
    if row['num_games'] < 20:
        return 0.1 * row['num_games'] + 0.5*row['wins_pct']
    elif 21 <= row['num_games'] < 150:
        return 0.15 * row['num_games'] + 1.5*row['wins_pct']
    else :
        return 0.15 * row['num_games'] + 0.75*row['wins_pct']

In [ ]:
# metric 3
team_stats['rs_metric_3'] = team_stats.apply(calc_rs_metric_by_bucket, axis=1)
team_stats_ranked = team_stats.sort_values(by ="rs_metric_3", ascending = False).reset_index(drop=True)
team_stats_ranked

,team_id,num_games,num_wins,num_loss,wins_pct,rs_metric,rs_metric_2,rs_metric_3
0,107581759470356115,88,75,13,85.23,85.9225,77.1225,141.0450
1,107603600826620492,78,62,16,79.49,79.1175,71.3175,130.9350
2,107598788926820994,79,62,17,78.48,78.6100,70.7100,129.5700
3,103535282158162659,135,98,37,72.59,88.1925,74.6925,129.1350
4,98767991853197861,512,334,178,65.23,176.9225,125.7225,125.7225
...,...,...,...,...,...,...,...,...
620,99124844362174905,1,0,1,0.00,0.2500,0.1500,0.1000
621,105352091319919029,1,0,1,0.00,0.2500,0.1500,0.1000
622,105301773561628001,1,0,1,0.00,0.2500,0.1500,0.1000
623,105351724613755142,1,0,1,0.00,0.2500,0.1500,0.1000
